In [1]:
import cv2
import os
import sys
import numpy as np
import pandas as pd

# Import library with current code functions
sys.path.append(os.path.join("..", "lib"))
import general_functions as gf, video_adjuster_functions as vid_adj_fun, files_paths as fp

In [2]:
MAIN_LIST_INFO =  gf.find_files_in_all_subdirectories([fp.DATASET_YT, fp.DATASET_LOCAL], fp.VD_INFO)

FILE_TO_INSERT_ANGLES = fp.VD_FEATURES_L2

## Extracting the X Y and Z angles for each pose of each frames and inserting at the last columns of the CSV

In [3]:
for i, current_path_location in enumerate(MAIN_LIST_INFO):

    path_dir = os.path.dirname(current_path_location)
    vd_feature_path = os.path.join(path_dir, FILE_TO_INSERT_ANGLES)
    video_feature_rest = pd.read_csv(vd_feature_path)

    print (str(i+1) + " of " + str(len(MAIN_LIST_INFO)) + ": " + vd_feature_path)

    # Remove the Unnamed columns
    if 'Unnamed: 0' in video_feature_rest.columns:
        video_feature_rest.drop(columns=['Unnamed: 0'], inplace=True)

    # Remove Rows with Nan Values
    video_feature_rest_drop_na = video_feature_rest.fillna(0)      
    
    POSE_FEATURE = pd.DataFrame()
    tvec = np.array([[0.0], [0.0], [0.0]])
    rvec = np.array([[0.0], [0.0], [0.0]])
    
    for row, row_data in video_feature_rest_drop_na.iterrows():
        if row_data.iloc[6]==0: 
            shape = {'rot_x': np.nan, 'rot_y': np.nan, 'rot_z': np.nan}
            video_extractor_pose = pd.DataFrame([shape])
            
        else:
            shape = vid_adj_fun.tuple_to_np(row_data[6:74])
            video_extractor_pose = pd.DataFrame()
            new_shape = vid_adj_fun.shape_normalizer(shape)
            
            refImgPts = vid_adj_fun.ref2dImagePoints(new_shape)
            image_width = (new_shape[16] - new_shape[0])[0]
            image_height = (new_shape[8] - new_shape[27])[1]
            angles, rvec, tvec = vid_adj_fun.get_angles(image_height, image_width, refImgPts, rotationVector = rvec, translationVector = tvec)
            video_extractor_pose['rot_x'] = [round(-angles[0],2)]
            video_extractor_pose['rot_y'] = [round(angles[1],2)]
            video_extractor_pose['rot_z'] = [round(-angles[2],2)]
            
        POSE_FEATURE = pd.concat([POSE_FEATURE, video_extractor_pose]).reset_index(drop=True)
    POSE_FEATURE = pd.concat([video_feature_rest, POSE_FEATURE], axis=1)

    base_path_folder_output = os.path.join(path_dir, FILE_TO_INSERT_ANGLES)
    POSE_FEATURE.to_csv(base_path_folder_output, na_rep='NaN')

1 of 70: ..\Dataset\DD-Local\VD_D_0000000043\VD_FEATURES_L2.CSV
2 of 70: ..\Dataset\DD-Local\VD_D_0000000044\VD_FEATURES_L2.CSV


KeyboardInterrupt: 